In [3]:
import pandas as pd

df = pd.read_csv('bike.csv')
df.head()


,instant,dteday,season,yr,mnth,holiday,weekday,temp,atemp,hum,windspeed,casual,registered,cnt,workday,weather,days_since_2011,cnt_2d_bfr
0,3,2011-01-03,WINTER,2011,JAN,N,MON,1.229108,22.439770,43.7273,16.636703,120,1229,1349,Y,GOOD,2,985
1,4,2011-01-04,WINTER,2011,JAN,N,TUE,1.400000,23.212148,59.0435,10.739832,108,1454,1562,Y,GOOD,3,801
2,5,2011-01-05,WINTER,2011,JAN,N,WED,2.666979,23.795180,43.6957,12.522300,82,1518,1600,Y,GOOD,4,1349
3,6,2011-01-06,WINTER,2011,JAN,N,THU,1.604356,23.929106,51.8261,6.000868,88,1518,1606,Y,GOOD,5,1562
4,7,2011-01-07,WINTER,2011,JAN,N,FRI,1.236534,23.100526,49.8696,11.304642,148,1362,1510,Y,MISTY,6,1600


In [4]:
import numpy as np

median_cnt = df['cnt'].median()
df['high_demand'] = (df['cnt'] > median_cnt).astype(int)
df = df.drop(columns='cnt')


In [5]:
#d)
# List of continuous features (example)
continuous_features = ['temp', 'atemp', 'hum', 'windspeed']

# Discretize using qcut into 4 quantiles
for col in continuous_features:
    df[col] = pd.qcut(df[col], q=4, duplicates='drop')

# One-hot encode
df = pd.get_dummies(df)

# Target
y = df['high_demand']
X = df.drop(columns='high_demand')


In [ ]:
#e
from sklearn.model_selection import train_test_split
from imodels import GreedyRuleListClassifier
from sklearn.metrics import accuracy_score

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model = GreedyRuleListClassifier()
model.fit(X_train, y_train)             

# Evaluate
train_acc = accuracy_score(y_train, model.predict(X_train))
test_acc = accuracy_score(y_test, model.predict(X_test))

print("Train Accuracy:", train_acc)
print("Test Accuracy:", test_acc)


Train Accuracy: 0.9518900343642611
Test Accuracy: 0.9315068493150684


In [7]:
print(model)


> ------------------------------
> Greedy Rule List
> ------------------------------
↓
16.21% risk (582 pts)
	if registered ==> 98.0% risk (255 pts)
↓
6.27% risk (327 pts)
	if casual ==> 87.5% risk (40 pts)
↓
0.0% risk (287 pts)
	if casual ==> 25.0% risk (72 pts)
↓
0.0% risk (215 pts)



### Interpreting Rules

- risk: Probability of misclassification within that rule’s coverage (1 - accuracy for that rule).
- pts: Number of data points covered by that rule.
- coverage: How many data points a rule applies to (related to pts).
- accuracy: 1 - risk. Higher accuracy means better performance of that rule.